In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wb_token = user_secrets.get_secret("Ming-Test")

In [3]:
wandb.login(key = wb_token)
run = wandb.init(
project = 'Fine-tune llama3.1 on sentiment analysis dataset',
job_type = 'training',
anonymous = 'allow')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vivaming. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240812_105335-udcdj75p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run spring-leaf-25
wandb: ⭐️ View project at https://wandb.ai/vivaming/Fine-tune%20llama3.1%20on%20sentiment%20analysis%20dataset
wandb: 🚀 View run at https://wandb.ai/vivaming/Fine-tune%20llama3.1%20on%20sentiment%20analysis%20dataset/runs/udc

In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-08-12 10:54:00.597580: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 10:54:00.597692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 10:54:00.708272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df = pd.read_csv("/kaggle/input/sentiment-analysis-for-mental-health-2/Combined Data.csv",index_col = "Unnamed: 0")

In [6]:
df.groupby('status').size().reset_index(name = 'count')

,status,count
0,Anxiety,3888
1,Bipolar,2877
2,Depression,15404
3,Normal,16351
4,Personality disorder,1201
5,Stress,2669
6,Suicidal,10653


In [7]:
df = df[(df.status != 'Personality disorder') & (df.status !='Stress') & (df.status !='Suicidal')]

In [8]:
df.groupby('status').size().reset_index(name = 'count')

,status,count
0,Anxiety,3888
1,Bipolar,2877
2,Depression,15404
3,Normal,16351


In [9]:
#shuffle the Dataframe and select only 3000 rows randomly

df = df.sample(frac = 1, random_state = 85).reset_index(drop = True).head(3000)

In [10]:
#split the Dataframe
train_size = 0.8
eval_size = 0.1

#calculate the size
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

#split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

In [11]:
# Define the prompt function

def generate_prompt(data_point):
    return f'''
    classify the text into Normal, Depression, Anxiety, Bipolar and return the answer as the corresponding mental health disorder label.
    text: {data_point["statement"]}
    label:data_point["status"]'''.strip()

def generate_test_prompt(data_point):
    return f'''
    classify the text into Normal, Depression, Anxiety, Bipolar and return the answer as the corresponding mental health disorder label.
    text: {data_point["statement"]}
    label:'''.strip()


In [12]:
# Generate prompt for training and evaluation data
X_train = X_train.copy()
X_train.loc[:, 'text'] = X_train.apply(generate_prompt, axis=1)

X_eval = X_eval.copy()  # Ensure you're working with a copy
X_eval.loc[:, 'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate prompts for the test data
y_true = X_test.loc[:, 'status']
X_test = X_test.copy()
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=['text'])


In [13]:
X_train.status.value_counts()

status
Normal        1028
Depression     938
Anxiety        258
Bipolar        176
Name: count, dtype: int64

In [14]:
X_eval.head()

,statement,status,text
2400,"A poem to me, by my bi-polar father. My dad di...",Bipolar,"classify the text into Normal, Depression, Anx..."
2401,Drowning in debt After being on the wrong meds...,Bipolar,"classify the text into Normal, Depression, Anx..."
2402,hi everyone can someone buy me meal i do not h...,Depression,"classify the text into Normal, Depression, Anx..."
2403,"Okay, I do not know where to start and it is g...",Depression,"classify the text into Normal, Depression, Anx..."
2404,Bipolar 2 and Sex Addiction (P&amp;M) Hi there...,Bipolar,"classify the text into Normal, Depression, Anx..."


In [15]:
#convert to dataset
train_data = Dataset.from_pandas(X_train[['text']])
eval_data = Dataset.from_pandas(X_eval[['text']])

In [16]:
train_data['text'][:3]

['classify the text into Normal, Depression, Anxiety, Bipolar and return the answer as the corresponding mental health disorder label.\n    text: The "Calm" I have Bipolar 2.\n\nAfter a month of isolation and depression, I went out with my friends last night and had a great time. I went home tired and slept the whole day. I woke up exhausted but the racing thoughts and irritability are gone. My mind is blank and surprisingly calm. I\'m actually panicking a bit because, at this point, I don\'t know what\'s happening. There are no more excessive intrusive thoughts, or those crazy ideas - just really calm. I feel alright but I still think of suicide.  I think what scares me a bit is that I experience this before a big mood swing. Am I okay? \n    label:data_point["status"]',
 'classify the text into Normal, Depression, Anxiety, Bipolar and return the answer as the corresponding mental health disorder label.\n    text: I have had a decent summer, nothing crazy like most kids but it was not

In [17]:
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [19]:
y_pred = []
test=X_test

categories = ["Normal", "Depression", "Anxiety", "Bipolar"]

# Print the first 3 records from X_test
for i in tqdm(range(len(test))):
    prompt = X_test.iloc[i]["text"]
    pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=2, 
                        temperature=0.1)
        
    result = pipe(prompt)
    input_text = result[0]["generated_text"].split("label:")[0]
    answer = result[0]["generated_text"].split("label:")[-1].strip()
    #print(f'{i+1} -- answer: {answer} - input Text: {input_text}')
    
    #Determine the predicted category
    for category in categories:
        if category.lower() in answer.lower():
            y_pred.append(category)
            break
    else:
        y_pred.append("none")
           



    

100%|██████████| 300/300 [02:55<00:00,  1.71it/s]


Evaluate the result

In [20]:
labels = ["Normal", "Depression", "Anxiety", "Bipolar"]
mapping = {label: idx for idx, label in enumerate(labels)}

def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

y_true_mapped = np.vectorize(map_func)(y_true)
y_pred_mapped = np.vectorize(map_func)(y_pred)

In [21]:
# Calculate accuracy
accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.797


In [22]:
# Generate accuracy report
unique_labels = set(y_true_mapped)  # Get unique labels
    
for label in unique_labels:
    label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
    label_y_true = [y_true_mapped[i] for i in label_indices]
    label_y_pred = [y_pred_mapped[i] for i in label_indices]
    label_accuracy = accuracy_score(label_y_true, label_y_pred)
    print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
        
# Generate classification report
class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
print('\nClassification Report:')
print(class_report)
    
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
print('\nConfusion Matrix:')
print(conf_matrix)

Accuracy for label Normal: 0.755
Accuracy for label Depression: 0.939
Accuracy for label Anxiety: 0.481
Accuracy for label Bipolar: 0.667

Classification Report:
              precision    recall  f1-score   support

      Normal       0.96      0.76      0.84       143
  Depression       0.71      0.94      0.81       115
     Anxiety       0.59      0.48      0.53        27
     Bipolar       0.91      0.67      0.77        15

   micro avg       0.80      0.80      0.80       300
   macro avg       0.79      0.71      0.74       300
weighted avg       0.83      0.80      0.80       300


Confusion Matrix:
[[108  29   5   0]
 [  2 108   4   1]
 [  3  11  13   0]
 [  0   4   0  10]]


# Extracting the linear modules names

In [23]:
import bitsandbytes as bnb

In [24]:
cls = bnb.nn.Linear4bit
lora_module_names = set()

for name, module in model.named_modules():
    print(f'Name: {name}')
    print(f'Module: {module}')
    print('-' * 50)  # A separator for clarity
    
    if isinstance(module, cls):
        names = name.split('.')
        lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        
if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')

modules =lora_module_names

Name: 
Module: LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNo

In [25]:
modules

{'down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj'}

https://www.kaggle.com/code/kingabzpro/fine-tune-llama-3-1-for-text-classification